In [1]:
# Data Manipulation
import pandas as pd
import numpy as np
import os

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Stats & Utilities
from scipy.stats import zscore, entropy, kurtosis
import networkx as nx
from itertools import product
from joblib import Parallel, delayed
from sklearn.utils import resample
from IPython.display import display

# Preprocessing
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, QuantileTransformer, OrdinalEncoder, RobustScaler, MaxAbsScaler
from sklearn.feature_extraction import FeatureHasher
from sklearn.neighbors import kneighbors_graph, NearestNeighbors

# Clustering
import hdbscan
from sklearn.cluster import KMeans, Birch, MiniBatchKMeans, SpectralClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, pairwise_distances
from hdbscan.validity import validity_index
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.mixture import GaussianMixture
from scipy.spatial.distance import pdist, squareform

# Dimensionality Reduction
import umap.umap_ as umap
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

#External Tests
from diptest import diptest

C:\Users\Eden Thomas\.conda\envs\thesis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Plotting Settings
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

In [3]:
# Load Dataset
file_path = "./Data/telecom_preprocessed.csv"
df = pd.read_csv(file_path)

# Preliminary Clustering

In [4]:
X = df.values

### KMEANS

In [122]:
k_values = range(2, 11)

# Store results
results = []

# Loop over k values
for k in k_values:
    try:
        km = KMeans(n_clusters=k, random_state=42, n_init='auto')
        labels = km.fit_predict(X)

        # Compute clustering metrics
        sil = silhouette_score(X, labels)
        dbi = davies_bouldin_score(X, labels)
        chi = calinski_harabasz_score(X, labels)

        # Store the results
        results.append({
            "k": k,
            "silhouette": sil,
            "dbi": dbi,
            "chi": chi
        })

        # Print live progress
        print(f"Done: k={k}, silhouette={sil:.3f}, dbi={dbi:.3f}, chi={chi:.1f}")

    except Exception as e:
        print(f"Error at k={k}: {e}")

# Final DataFrame
df_kmeans_results = pd.DataFrame(results).sort_values(by="silhouette", ascending=False).reset_index(drop=True)
display(df_kmeans_results)

Done: k=2, silhouette=0.366, dbi=1.092, chi=24488.3
Done: k=3, silhouette=0.388, dbi=1.167, chi=31712.7
Done: k=4, silhouette=0.143, dbi=1.878, chi=25450.6
Done: k=5, silhouette=0.151, dbi=1.770, chi=22034.6
Done: k=6, silhouette=0.179, dbi=1.772, chi=19301.4
Done: k=7, silhouette=0.155, dbi=2.201, chi=16848.0
Done: k=8, silhouette=0.177, dbi=1.804, chi=17358.8
Done: k=9, silhouette=0.184, dbi=1.698, chi=17018.2
Done: k=10, silhouette=0.190, dbi=1.723, chi=15938.0


,k,silhouette,dbi,chi
0,3,0.388236,1.167146,31712.692571
1,2,0.366429,1.092433,24488.316471
2,10,0.190203,1.722751,15937.973971
3,9,0.184006,1.698382,17018.155014
4,6,0.179387,1.771534,19301.385009
5,8,0.177162,1.804221,17358.765910
6,7,0.154692,2.200938,16848.014848
7,5,0.151476,1.770284,22034.584000
8,4,0.143379,1.877659,25450.640621


### GMM

In [123]:
# k values to test
k_values = range(2, 11)

# Store results
results = []

# Loop over k values
for k in k_values:
    try:
        gmm = GaussianMixture(n_components=k, random_state=42)
        labels = gmm.fit_predict(X)

        # Compute clustering metrics
        sil = silhouette_score(X, labels)
        dbi = davies_bouldin_score(X, labels)
        chi = calinski_harabasz_score(X, labels)

        # Store result
        results.append({
            "k": k,
            "silhouette": sil,
            "dbi": dbi,
            "chi": chi
        })

        # Print live progress
        print(f"Done: k={k}, silhouette={sil:.3f}, dbi={dbi:.3f}, chi={chi:.1f}")

    except Exception as e:
        print(f"Error at k={k}: {e}")

# Final DataFrame
df_gmm_results = pd.DataFrame(results).sort_values(by="silhouette", ascending=False).reset_index(drop=True)
display(df_gmm_results)

Done: k=2, silhouette=0.085, dbi=2.601, chi=12094.8
Done: k=3, silhouette=0.168, dbi=1.888, chi=20794.8
Done: k=4, silhouette=0.105, dbi=2.738, chi=20502.7
Done: k=5, silhouette=0.095, dbi=3.135, chi=15525.9
Done: k=6, silhouette=0.160, dbi=3.189, chi=14179.0
Done: k=7, silhouette=0.081, dbi=5.094, chi=12186.8
Done: k=8, silhouette=0.112, dbi=2.983, chi=13332.1
Done: k=9, silhouette=0.153, dbi=3.037, chi=12526.3
Done: k=10, silhouette=0.157, dbi=3.145, chi=11522.7


,k,silhouette,dbi,chi
0,3,0.167749,1.888094,20794.775715
1,6,0.160119,3.188712,14178.967340
2,10,0.157189,3.144510,11522.695494
3,9,0.152631,3.037305,12526.257056
4,8,0.111740,2.983276,13332.105234
5,4,0.104951,2.738328,20502.723460
6,5,0.094837,3.134671,15525.855182
7,2,0.085355,2.600636,12094.750477
8,7,0.081061,5.094348,12186.783943


### BIRCH

In [141]:
# Manual threshold input
BIRCH_THRESHOLD = 5

# Run BIRCH
birch = Birch(n_clusters=None, threshold=BIRCH_THRESHOLD)
labels_birch = birch.fit_predict(X)
subcentres = birch.subcluster_centers_
n_sub = subcentres.shape[0]

print(f"Birch threshold={BIRCH_THRESHOLD} produced {n_sub} subclusters.\n")

Birch threshold=5 produced 2844 subclusters.



In [142]:
# KMeans on BIRCH subcluster centers
results = []
k_values = range(2, 11)

for k in k_values:
    try:
        km = KMeans(n_clusters=k, random_state=42, n_init='auto')
        labels = km.fit_predict(subcentres)

        sil = silhouette_score(subcentres, labels)
        dbi = davies_bouldin_score(subcentres, labels)
        chi = calinski_harabasz_score(subcentres, labels)

        results.append({
            "k": k,
            "silhouette": sil,
            "dbi": dbi,
            "chi": chi
        })

        print(f"KMeans on subclusters: k={k}, silhouette={sil:.3f}, dbi={dbi:.3f}, chi={chi:.1f}")
    
    except Exception as e:
        print(f"Error at k={k}: {e}")

# Final table
df_birch_results = pd.DataFrame(results).sort_values(by="silhouette", ascending=False).reset_index(drop=True)
display(df_birch_results)

KMeans on subclusters: k=2, silhouette=0.137, dbi=2.201, chi=426.8
KMeans on subclusters: k=3, silhouette=0.193, dbi=1.749, chi=666.1
KMeans on subclusters: k=4, silhouette=0.178, dbi=1.642, chi=608.5
KMeans on subclusters: k=5, silhouette=0.152, dbi=1.803, chi=543.6
KMeans on subclusters: k=6, silhouette=0.154, dbi=1.915, chi=493.2
KMeans on subclusters: k=7, silhouette=0.158, dbi=1.865, chi=447.5
KMeans on subclusters: k=8, silhouette=0.145, dbi=1.897, chi=404.7
KMeans on subclusters: k=9, silhouette=0.154, dbi=1.870, chi=384.8
KMeans on subclusters: k=10, silhouette=0.142, dbi=1.889, chi=365.5


,k,silhouette,dbi,chi
0,3,0.192648,1.748562,666.149472
1,4,0.178383,1.642202,608.528990
2,7,0.157988,1.864586,447.473952
3,6,0.153557,1.914905,493.161180
4,9,0.153556,1.869777,384.835367
5,5,0.151972,1.803499,543.596289
6,8,0.145179,1.897077,404.686881
7,10,0.141654,1.888857,365.510357
8,2,0.137405,2.201252,426.775495


### HDBSCAN

In [ ]:
# CONFIG
SAMPLE_SIZE = 90_000            # 30_000 rows from 100_000
RANDOM_STATE = 42             # for reproducibility
N_JOBS = -1                 # use all cores

# Parameter grid
param_grid = {
    "min_cluster_size": [250,300, 350, 400, 450, 500, 550, 600, 650,700,750,800],
    "min_samples": [None],
}

#Sample for preliminary analysis
if X.shape[0] > SAMPLE_SIZE:
    rng = np.random.default_rng(42)
    sample_idx = rng.choice(X.shape[0], SAMPLE_SIZE, replace=False)
    Xs = X[sample_idx]
else:
    Xs = X

# Define function to run one HDBSCAN fit
def run_hdbscan(mcs, ms):
    hdb = hdbscan.HDBSCAN(
        min_cluster_size=mcs,
        min_samples=ms,
        metric="euclidean",
        core_dist_n_jobs=-1,
        approx_min_span_tree=True,
        prediction_data=False,
    )
    labels = hdb.fit_predict(Xs)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

    # Quality scores
    sil = (silhouette_score(Xs, labels) if n_clusters >= 2 else np.nan)
    mask = labels != -1
    if mask.sum() >= 2 and len(np.unique(labels[mask])) >= 2:
        dbi = davies_bouldin_score(Xs[mask], labels[mask])
        chi = calinski_harabasz_score(Xs[mask], labels[mask])
    else:
        dbi, chi = np.nan, np.nan

    return {
        "min_cluster_size": mcs,
        "min_samples": ms,
        "k": n_clusters,
        "silhouette": sil,
        "dbi": dbi,
        "chi": chi,
    }

# Grid Search
combos = [(mcs, ms) for mcs in param_grid["min_cluster_size"] 
                     for ms  in param_grid["min_samples"]]
results = Parallel(n_jobs=N_JOBS)(
    delayed(run_hdbscan)(mcs, ms) for mcs, ms in combos
)

# Results
df = pd.DataFrame(results)
display(df)